In [12]:
import numpy as np
import scipy.sparse as sp
import tensorflow as tf

In [13]:
path="./data/cora/"
dataset="cora"

In [18]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [19]:
idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                    dtype=np.dtype(str))
labels = encode_onehot(idx_features_labels[:, -1])

In [20]:
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
idx_map = {j: i for i, j in enumerate(idx)}

In [21]:
edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
edges_transpose = np.transpose(edges)

In [22]:
A = np.zeros(shape=(idx.shape[0],idx.shape[0]))
for e in edges:
    A[e[0]][e[1]]=1
X = idx_features_labels[:, 1:-1]
Y=labels

In [23]:
D=np.eye(idx.shape[0])
I=np.eye(idx.shape[0])
for e in edges:
    D[e[0]][e[0]]=np.sum(edges_transpose[0] == e[0])
AI = A + I
D_inv = np.linalg.inv(D)
D_half = np.dot(D**0.5,D_inv)
A_t = np.dot(np.dot(D_half,AI),D_half)

In [24]:
# Deg[i] means the no of edges terminating on that vertex ie np.sum(edges_transpose[0]==i)
# D is diag(Deg[i]) $\forall$ i
#AI = A + I .. defining that a node i is connected to itself

In [25]:
# X is N X D , Y is N X K , A_t is A X N .. good to go, we need degree matrix Deg

In [26]:
print A_t.shape , X.shape

(2708, 2708) (2708, 1433)


In [27]:
F = 16 # the dimensions of latent variable
H = 32 # the dimension for Hidden unit
D = X.shape[1]
# W0 would be D X H, W1 would be H X F

In [29]:
w0 = tf.Variable(tf.random_uniform([D, H]),trainable=True)
w1 = tf.Variable(tf.random_uniform([H, F]),trainable=True)
A_t_tf = tf.convert_to_tensor(A_t, dtype=tf.float32)
AI_tf = tf.convert_to_tensor(AI, dtype=tf.float32)
X_tf = tf.convert_to_tensor(X,dtype=tf.float32)

In [30]:
# First layer :
z1 = tf.matmul(tf.matmul(A_t_tf,X_tf),w0) 
a1 = tf.nn.relu(z1)

In [31]:
#Second Layer :
z2 = tf.matmul(tf.matmul(A_t_tf,a1),w1)
# a2 = tf.nn.softmax(z2)

#Final assignent to Z, it has to be N X F
Z = z2

In [48]:
#defining the loss function : 
mat = tf.matmul(Z,tf.transpose(Z))
mat_flat=tf.reshape(mat, [-1])
A_t_tf_flat = tf.reshape(A_t_tf,[-1])
AI_tf_flat = tf.reshape(AI_tf,[-1]) 
n = X.shape[0]**2 
ne = edges.shape[0]
ps = (n**2 - ne)/ne
norm = n**2/float((n**2-ne)**2)
loss = norm*tf.reduce_sum(tf.nn.weighted_cross_entropy_with_logits(targets=AI_tf_flat,logits=mat_flat,pos_weight=ps))

In [49]:
opt = tf.train.AdamOptimizer(0.01).minimize(loss)

In [50]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(200):
        print(sess.run([loss]))
        sess.run(opt)

[0.24762732]
[0.22900866]
[0.21146204]
[0.19498457]
[0.17955399]
[0.16514073]
[0.15171061]
[0.13922614]
[0.12764716]
[0.11693174]
[0.10703638]
[0.0979169]
[0.089528635]
[0.081826955]
[0.07476768]
[0.06830743]
[0.062403977]
[0.057016447]
[0.05210557]
[0.04763385]
[0.043565746]
[0.03986768]
[0.036508095]
[0.03345752]
[0.030688547]
[0.0281758]
[0.025895804]
[0.023826957]
[0.021949437]
[0.020245122]
[0.018697483]
[0.017291496]
[0.016013503]
[0.014851105]
[0.013793075]
[0.012829293]
[0.01195059]
[0.0111487135]
[0.010416216]
[0.009746385]
[0.009133206]
[0.008571246]
[0.008055624]
[0.0075819436]
[0.0071462668]
[0.0067450292]
[0.006375037]
[0.006033408]
[0.005717563]
[0.005425167]
[0.005154132]
[0.0049025514]
[0.0046687215]
[0.004451103]
[0.0042483043]
[0.0040590637]
[0.0038822412]
[0.0037168209]
[0.003561861]
[0.0034165233]
[0.0032800282]
[0.0031516752]
[0.0030308336]
[0.0029169281]
[0.0028094356]
[0.002707879]
[0.0026118206]
[0.002520863]
[0.0024346479]
[0.0023528435]
[0.0022751396]
[0.00220

In [43]:
# x = tf.Variable(10.0, trainable=True)
# y = tf.Variable(10.0, trainable=True)


# fx = tf.nn.l2_loss(tf.exp(x)+tf.multiply(x,y)-20)
# loss = fx
# opt = tf.train.GradientDescentOptimizer(0.000002).minimize(fx)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(200):
#         print(sess.run([x,y,loss]))
#         sess.run(opt)